In [2]:
import pandas as pd
import numpy as np
import json
import warnings

warnings.filterwarnings(action='ignore')

data_dir = './Data/'

# 데이터 확인

In [3]:
song_meta_df = pd.read_json(data_dir + 'song_meta_data_v3.json')
song_meta_df = song_meta_df.sort_values('id')
song_meta_df = song_meta_df.reset_index(drop = True)
song_meta_df['song_embedding_idx'] = song_meta_df.index

song_id = song_meta_df['id'].tolist()
song_embedding_idx = song_meta_df['song_embedding_idx'].tolist()

song_id2song_embedding_idx = {}
song_embedding_idx2song_id = {}
for id, idx in zip(song_id, song_embedding_idx):
    song_id2song_embedding_idx[id] = idx
    song_embedding_idx2song_id[idx] = id

word2vec_embedding = np.load(data_dir + 'word2vec_embedding.npy')
mel_embedding = np.load(data_dir + 'tanh_mel_embeding_loss55.npy')
cos_embedding = np.load(data_dir + 'get_cos_embedding_63.npy')

In [4]:
with open(data_dir + 'top10_song_id2playlist.json', 'r') as f:
    top10_song_id2playlist = json.load(f)
    
with open(data_dir + 'ranking_song_id2playlist.json', 'r') as f:
    ranking_song_id2playlist = json.load(f)

# 힘수 정의

In [21]:
def get_cos_sim(song_id, embedding, song_id2song_embedding_idx, song_embedding_idx2song_id, top = 10):
    
    song_embedding_idx = song_id2song_embedding_idx[song_id]

    dot_ret = np.sum(embedding[song_embedding_idx].reshape(1, -1) * embedding, axis=1)

    norm_ret = np.linalg.norm(embedding[song_embedding_idx]) * np.linalg.norm(embedding, axis=1)

    ret = dot_ret / norm_ret

    sorted_idx_ret = np.argsort(ret)[::-1]

    ret_song_id_li = [song_embedding_idx2song_id[idx] for idx in sorted_idx_ret[:top]]
    
    return ret_song_id_li

def see_data(data, song_id_li):
    cols = ['song_name', 'artist_name_basket', 'kor_song_gn_gnr_basket', 'like', 'album_name']
    display(data.set_index('id').loc[song_id_li, cols])

# 서빙

In [41]:
song_id = 166841

In [40]:
song_meta_df[song_meta_df['song_name'].str.contains('어떻게 이별까지 사랑하겠어, 널 사랑하는 거지')]

,id,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,like,kor_song_gn_gnr_basket,song_embedding_idx
39805,166841,"[GN0105, GN0101]",20190925,항해,10331947,[712452],"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",[GN0100],[AKMU (악동뮤지션)],0,[발라드],39805


In [42]:
# word2vec
see_data(data = song_meta_df, song_id_li = get_cos_sim(song_id = song_id, embedding = word2vec_embedding, song_id2song_embedding_idx = song_id2song_embedding_idx, song_embedding_idx2song_id = song_embedding_idx2song_id, top = 10))

,song_name,artist_name_basket,kor_song_gn_gnr_basket,like,album_name
id,,,,,
166841,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",[AKMU (악동뮤지션)],[발라드],0,항해
454890,마음,[폴킴],[R&B/Soul],78200,마음
621320,편의점 앞에서,[김진호 (SG워너비)],[발라드],1534,노래샘
114092,가을이 오면,[레이나 (Raina)],[발라드],275,Project 10
41912,사계 (Four Seasons),[태연 (TAEYEON)],"[아이돌, 발라드]",239527,사계 (Four Seasons)
560941,단 한 번 (Feat. KEM),[A Song],[발라드],92,단 한 번
92330,With You,"[마크툽 (MAKTUB), 이라온]","[OST, 발라드]",2639,조선혼담공작소 꽃파당 OST Part.4
145750,그려본다,[반광옥],[발라드],3374,그려본다
236114,Done For Me,[펀치 (Punch)],"[OST, 발라드]",98082,호텔 델루나 OST


In [43]:
# mel
see_data(data = song_meta_df, song_id_li = get_cos_sim(song_id = song_id, embedding = mel_embedding, song_id2song_embedding_idx = song_id2song_embedding_idx, song_embedding_idx2song_id = song_embedding_idx2song_id, top = 10))

,song_name,artist_name_basket,kor_song_gn_gnr_basket,like,album_name
id,,,,,
166841,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",[AKMU (악동뮤지션)],[발라드],0,항해
433688,어떻게 말할까 (Acoustic Ver.),[오왠 (O.WHEN)],[OST],493,수상한 파트너 OST
588039,남촌 (Piano Ver.),[김가온],[재즈],20,Prismatic
616261,잘지내니,[멜로우스푼],"[인디음악, 포크/블루스]",1662,잘지내니
528362,White Christmas,"[Bing Crosby, Frank Sinatra]",[POP],19,Merry Christmas
619645,Sign of the Times,"[Jasmine Thompson, Sabrina Carpenter]",[POP],3663,Sign of the Times
347842,My Romance,[김동률],[발라드],432,"2008 Concert, Monologue"
334542,라일라,[브로큰 펄],[록/메탈],67,Way Out Vol.1
198883,Time To Grow (Acoustic Piano Ver.) (Bonus Track),[루싸이트 토끼],"[인디음악, 록/메탈]",97,Grow To Glow


In [44]:
# cos
see_data(data = song_meta_df, song_id_li = get_cos_sim(song_id = song_id, embedding = cos_embedding, song_id2song_embedding_idx = song_id2song_embedding_idx, song_embedding_idx2song_id = song_embedding_idx2song_id, top = 10))

,song_name,artist_name_basket,kor_song_gn_gnr_basket,like,album_name
id,,,,,
166841,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",[AKMU (악동뮤지션)],[발라드],0,항해
413837,만약에 말야 (전우성 Solo),[노을],[발라드],90748,TIME FOR LOVE
472428,월급 받던 날,[포스트맨 (Postmen)],[발라드],1030,서툰 고백
32698,다시 내게,[권영찬],"[인디음악, 발라드]",552,연주회
192349,나만 시작한다면,[이오공감],[발라드],546,25공감 (共感)
675953,Suzanne,[Judy Collins],[포크/블루스/컨트리],116750,Rhino Hi-Five : Judy Collins
72499,처음처럼,[엠씨더맥스 (M.C the MAX)],[발라드],108017,CEREMONIA
170555,One Of These Days,[Pink Floyd],[록/메탈],46,Echoes : The Best of Pink Floyd
560941,단 한 번 (Feat. KEM),[A Song],[발라드],92,단 한 번


In [45]:
# top10
see_data(data = song_meta_df, song_id_li = top10_song_id2playlist[str(song_id)])

,song_name,artist_name_basket,kor_song_gn_gnr_basket,like,album_name
id,,,,,
616261,잘지내니,[멜로우스푼],"[인디음악, 포크/블루스]",1662,잘지내니
588039,남촌 (Piano Ver.),[김가온],[재즈],20,Prismatic
621320,편의점 앞에서,[김진호 (SG워너비)],[발라드],1534,노래샘
454890,마음,[폴킴],[R&B/Soul],78200,마음
114092,가을이 오면,[레이나 (Raina)],[발라드],275,Project 10
413837,만약에 말야 (전우성 Solo),[노을],[발라드],90748,TIME FOR LOVE
472428,월급 받던 날,[포스트맨 (Postmen)],[발라드],1030,서툰 고백
433688,어떻게 말할까 (Acoustic Ver.),[오왠 (O.WHEN)],[OST],493,수상한 파트너 OST
166841,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",[AKMU (악동뮤지션)],[발라드],0,항해


In [46]:
# ranking
see_data(data = song_meta_df, song_id_li = ranking_song_id2playlist[str(song_id)])

,song_name,artist_name_basket,kor_song_gn_gnr_basket,like,album_name
id,,,,,
166841,"어떻게 이별까지 사랑하겠어, 널 사랑하는 거지",[AKMU (악동뮤지션)],[발라드],0,항해
413837,만약에 말야 (전우성 Solo),[노을],[발라드],90748,TIME FOR LOVE
454890,마음,[폴킴],[R&B/Soul],78200,마음
433688,어떻게 말할까 (Acoustic Ver.),[오왠 (O.WHEN)],[OST],493,수상한 파트너 OST
588039,남촌 (Piano Ver.),[김가온],[재즈],20,Prismatic
621320,편의점 앞에서,[김진호 (SG워너비)],[발라드],1534,노래샘
472428,월급 받던 날,[포스트맨 (Postmen)],[발라드],1030,서툰 고백
560941,단 한 번 (Feat. KEM),[A Song],[발라드],92,단 한 번
616261,잘지내니,[멜로우스푼],"[인디음악, 포크/블루스]",1662,잘지내니
